%load_ext autoreload
%autoreload 2

In [16]:
import pandas as pd
import numpy as np
import sys
sys.path.append('./')
import xiadan_check
import myModule
import imp
imp.reload(xiadan_check)
imp.reload(myModule)
import time

In [17]:
pathway = './'
all_files  = xiadan_check.get_files(pathway, "个性化","9月")   ##给路径
df1  = xiadan_check.read_xlsx(all_files , 0 )## read all files sheet1

In [18]:
import datetime
def changetype1(check_str):
    value = 1 if type(check_str) == datetime.datetime else 0
    return value
def changetype2(check_str):
    value = 0 if type(check_str) == datetime.datetime else 1
    return value

In [19]:
date = pd.to_datetime(time.strftime('%Y-%m-%d',time.localtime(time.time())))
stat_week = '9月4周'
## df1:原始项目；
## df2:有效反馈项目，即排除驳回、暂停、沟通中
## df3:在线项目，基于df2
## df4:完成项目，基于df2
## df5:延期项目，基于df3中未交付延期、df4

myModule.add_week(df1,'下单时间', 3)                                              #以下单时间定义week_number
myModule.upgrade_df(df1, [['反馈周期', ['评估反馈时间','下单时间'], '-']])     #增加“反馈周期”列
tt = lambda x : getattr(x , 'days')                                            #更换“days”类型
[myModule.retype(df1, i , tt) for i in ['反馈周期']] 

df2 = df1[df1['预计交付时间'].apply(lambda x: type(x) == datetime.datetime)]   #根据“预计交付时间”筛选有效反馈项目列表
df2['预计交付时间'] = df2['预计交付时间'].apply(pd.to_datetime)                #转换时间类型
myModule.upgrade_df(df2, [['预计周期', ['预计交付时间','评估反馈时间'], '-']]) #增加“预计周期”列
[myModule.retype(df2, i , tt) for i in ['预计周期']]                           #更换“days”类型

df3 = df2[df2['实际交付时间'].apply(lambda x: x == 0)]                          #根据“实际交付时间”筛选在线项目列表

df4 = df2[df2['实际交付时间'].apply(lambda x: type(x) == datetime.datetime)]    #根据“实际交付时间”筛选已交付项目列表
df4['实际交付时间'] = df4['实际交付时间'].apply(pd.to_datetime)                 #转换时间类型
myModule.upgrade_df(df4, [['实际周期', ['实际交付时间','评估反馈时间'], '-'],   #增加“实际周期”、“延期天数”列
                         ['延期天数',['实际交付时间','预计交付时间'], '-']])
[myModule.retype(df4, i , tt) for i in ['实际周期','延期天数']]                 #更换“days”类型
myModule.add_week(df4,'实际交付时间',3)                                           #以实际交付时间定义week_number

tmp = df3[df3['预计交付时间'].apply(lambda x: date>x)]                          #未交付延期项目
tmp['实际周期'] = 1000
tmp['延期天数'] = 1000
df5 = df4.append(tmp)
myModule.add_week(df5,'预计交付时间',3)                                           #以预计交付时间定义week_number

## 分析组人员列表
fenxi = ["施毓芷","闻少楠","杨宇","刘永强","刘梦苑","何胜","陈碧雯","王霞","卢楠","冯敬岩","朱慧萍","赵倩","解飞","宋少宇"]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

/mnt/e/jupyter/bin/supplment_analysis/myModule.py:87: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

/mnt/e/jupyter/bin/supplment_analysis/myModule.py:129: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

/usr/lo

# 评估情况统计

In [20]:
df1.head()

,序号,流水号,个性化分析名称,项目管理,接入时间,下单时间,信息负责人,评估反馈时间,预计交付时间,数据迁回花费时间（天）,实际交付时间,评估人时（h）,评估机时(h),是否在库里,库里周期（天）,项目管理交付时间,week_number,反馈周期
0,961,20180902-001,PM-GD180463-01华南农业大学6个青枯菌原核链特异性转录组建库测序过滤任务单-02,孟凡姗,2018-09-02,2018-09-02,屈昂,2018-09-05,2018-09-10 00:00:00,0,2018-09-07 00:00:00,3,18.0,0,0.0,2018-09-07 00:00:00,8月6周,3
1,962,20180902-002,ANCSN180125西安交大数据分析任务单-03,孟凡姗,2018-09-02,2018-09-02,朱慧萍,2018-09-03,2018-09-10 00:00:00,0,2018-09-06 00:00:00,0,0.0,0,0.0,2018-09-07 00:00:00,8月6周,1
2,963,20180902-003,ANCSN180125西安交大数据分析任务单-04,孟凡姗,2018-09-02,2018-09-02,朱慧萍,2018-09-03,2018-09-10 00:00:00,0,2018-09-06 00:00:00,0,0.0,0,0.0,2018-09-07 00:00:00,8月6周,1
3,964,20180902-004,PM-JS180194-01通大附院18个大鼠有参转录组建库测序分析任务单-快速产品-02,孟凡姗,2018-09-02,2018-09-02,朱慧萍,2018-09-03,2018-09-10 00:00:00,0,2018-09-10 00:00:00,0,0.0,0,0.0,2018-09-10 00:00:00,8月6周,1
4,965,20180902-005,PM-JS180194-01通大附院18个大鼠有参转录组建库测序分析任务单-快速产品-03,孟凡姗,2018-09-02,2018-09-02,朱慧萍,2018-09-03,2018-09-10 00:00:00,0,2018-09-10 00:00:00,0,0.0,0,0.0,2018-09-10 00:00:00,8月6周,1


## 接入项目数统计

### 整体接入项目数趋势图

In [21]:
ss = myModule.format(df1,'week_number',['流水号','个性化分析名称','项目管理','预计交付时间'])
ss.df['接入项目数'] = list(map(changetype2,ss.df['流水号']))
ss.df['有效接入项目数'] = list(map(changetype1,ss.df['预计交付时间']))
ss.df['驳回暂停沟通项目数'] = list(map(changetype2,ss.df['预计交付时间']))
tt = myModule.format(ss.df,'week_number',['有效接入项目数','驳回暂停沟通项目数','接入项目数'])
pp = myModule.sort_by_index(tt.groupby('sum'))
myModule.myPlot(pp).plot('line')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-ve

### 个人接入项目数分布图

In [22]:
ss = myModule.format(df1 ,'week_number' ,['流水号'])
ss.add_group('信息负责人')
pp = myModule.sort_by_index(ss.groupby('count'))
myModule.myPlot(pp).plot('stack_bar')

## 评估周期情况统计

### 预计周期趋势图

In [23]:
# 平均评估周期，建议添加最长评估周期、25%、50%、80%项目评估周期的指标
ss = myModule.format(df2 ,'week_number' ,['预计周期'])
pp = myModule.sort_by_index(ss.groupby('mean'))
myModule.myPlot(pp).plot('line')

### 预计周期>7日项目列表

In [24]:
ss = myModule.format(df2,'week_number',['流水号','个性化分析名称','信息负责人','预计周期'])
ss.select_by_value('week_number',stat_week)
ss.judge_and_replace('预计周期',7)
ss.select_by_value('预计周期_bool',True)
tt = myModule.format(ss.df,'week_number',['流水号','个性化分析名称','信息负责人','预计周期']).df
tt.sort_values(by = '预计周期',axis = 0,ascending = False)   #预计周期从大到小排序

,week_number,流水号,个性化分析名称,信息负责人,预计周期
182,9月4周,20180928-003,PM-YWYN170094-16云南省第一人民医院5个人外显子建库测序分析任务单-快速产品-...,陈碧雯,18
195,9月4周,20180930-003,PM-SD180234-01山东省立医院78个人肿瘤外显子建库测序自动化分析任务单-01,封彦杰,16
194,9月4周,20180930-002,PM-GD180479-01中山大学6个人有参普通转录组建库测序分析任务单-快速产品-02,杨宇,15
178,9月4周,20180927-006,PM-ZJ180327-01浙医二院35个人普通转录组建库测序过滤任务单-快速产品-01,屈昂,14
179,9月4周,20180927-007,BJ180998-02中药所1个药用动物S4基因组denovo只分析任务单,马潇,14
193,9月4周,20180930-001,SD170077-01山东大学齐鲁医院王刚的6个人单基因病外显子建库测序分析任务单-01,解飞,14
188,9月4周,20180929-002,PM-BJ181242-03中国农大一个薏米Hi-C辅助基因组组装建库测序分析任务单-加急-02,刘梦苑,13
175,9月4周,20180927-003,BJ170165-02中科院动物所10个小鼠单细胞甲基化建库任务单-04,韩少怀,12
185,9月4周,20180928-006,PM-BJ180911-01中国农业科学院北京畜牧所12个牛单细胞全转录组建库测序分析任务单...,冯敬岩,12
187,9月4周,20180929-001,PM-BJ181079-02动物所32个小鼠ChIP-DNA建库测序分析任务单-01,昶吉阳,12


## 反馈延期情况统计

### 反馈延期率趋势图

In [25]:
ss = myModule.format(df1 ,'week_number' ,['流水号','反馈周期'])
ss.judge_and_replace('反馈周期',1)
ss.df['流水号'] = list(map(changetype2, ss.df['流水号']))
ss.df['反馈周期_bool'] = ss.df['反馈周期_bool'].astype(int)      #bool类型装换为数值1/0
pp = myModule.sort_by_index(ss.get_total_ratio([['反馈延期率' ,'反馈周期_bool' , '流水号','/']]))
myModule.myPlot(pp).plot('line')

/mnt/e/jupyter/bin/supplment_analysis/myModule.py:246: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

### 反馈延期项目列表

In [26]:
ss = myModule.format(df1,'week_number',['流水号','个性化分析名称','信息负责人','下单时间','评估反馈时间','反馈周期'])
ss.select_by_value('week_number',stat_week)
ss.judge_and_replace('反馈周期',1)
ss.select_by_value('反馈周期_bool',True)
tt = myModule.format(ss.df,'week_number',['流水号','个性化分析名称','信息负责人','下单时间','评估反馈时间','反馈周期']).df
tt.sort_values(by = '反馈周期',axis = 0,ascending = False)

,week_number,流水号,个性化分析名称,信息负责人,下单时间,评估反馈时间,反馈周期
190,9月4周,20180929-005,PM-GD170317-01深圳市人民医院16个人6mA建库测序分析任务单-01,宋少宇,2018-09-29,2018-10-09,10
178,9月4周,20180927-006,PM-ZJ180327-01浙医二院35个人普通转录组建库测序过滤任务单-快速产品-01,屈昂,2018-09-27,2018-09-30,3
185,9月4周,20180928-006,PM-BJ180911-01中国农业科学院北京畜牧所12个牛单细胞全转录组建库测序分析任务单...,冯敬岩,2018-09-28,2018-09-30,2


## 在线项目数统计

In [27]:
ss = myModule.format(df3 ,'信息负责人' ,['流水号'])
pp = ss.groupby('count')
myModule.myPlot(pp).plot('bar')

# 交付情况统计

## 完成项目数统计

### 整体完成项目数趋势图

In [28]:
ss = myModule.format(df4 ,'week_number' ,['实际交付时间'])
pp = myModule.sort_by_index(ss.groupby('count'))
myModule.myPlot(pp).plot('line')

### 个人完成项目数统计

In [29]:
ss = myModule.format(df4 ,'week_number' ,['实际交付时间'])
ss.add_group('信息负责人')
pp = myModule.sort_by_index(ss.groupby('count'))
myModule.myPlot(pp).plot('stack_bar')

## 完成周期统计

### 整体完成周期趋势图

In [30]:
# 实际完成平均周期，建议添加最长完成周期、25%、50%、80%项目完成周期的指标
ss = myModule.format(df4 ,'week_number' ,['实际周期'])
pp = myModule.sort_by_index(ss.groupby('mean'))
myModule.myPlot(pp).plot('line')

### 个人完成周期统计

In [31]:
df4_fx = df4[df4['信息负责人'].isin(fenxi)] 
ss = myModule.format(df4_fx,'信息负责人' ,['实际周期'])
pp = ss.groupby('mean')
myModule.myPlot(pp).plot('bar')

### 实际周期>7日项目列表

In [32]:
# 实际周期>7日项目列表
ss = myModule.format(df4,'week_number',['流水号','个性化分析名称','信息负责人','评估反馈时间','预计交付时间','实际交付时间','实际周期'])
ss.select_by_value('week_number',stat_week)
ss.judge_and_replace('实际周期',7)
ss.select_by_value('实际周期_bool',True)
tt = myModule.format(ss.df,'week_number',['流水号','个性化分析名称','信息负责人','评估反馈时间','预计交付时间','实际交付时间','实际周期']).df
tt.sort_values(by = '实际周期',axis = 0,ascending = False)

,week_number,流水号,个性化分析名称,信息负责人,评估反馈时间,预计交付时间,实际交付时间,实际周期
96,9月4周,20180914-008,PM-NM180047-01内蒙古大学9个单细胞转录组建库测序分析-03,赵倩,2018-09-17,2018-09-27,2018-09-27,10
113,9月4周,20180918-005,YWZJ180310-01浙大昆虫所21个褐飞虱无参普通转录组只分析任务单-01,王霞,2018-09-19,2018-09-26,2018-09-27,8


# 延期情况统计

## 延期率趋势图

In [33]:
ss = myModule.format(df5 ,'week_number' ,['流水号','延期天数'])
ss.judge_and_replace('延期天数',1)
ss.df['流水号'] = list(map(changetype2, ss.df['流水号']))
ss.df['延期天数_bool'] = ss.df['延期天数_bool'].astype(int)      #bool类型装换为数值1/0
pp = myModule.sort_by_index(ss.get_total_ratio([['项目延期率' ,'延期天数_bool' , '流水号','/']]))
myModule.myPlot(pp).plot('line')

/mnt/e/jupyter/bin/supplment_analysis/myModule.py:246: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

## 延期项目列表

In [34]:
ss = myModule.format(df5,'week_number',['流水号','个性化分析名称','信息负责人','评估反馈时间','预计交付时间','实际交付时间','延期天数'])
ss.select_by_value('week_number',stat_week)
ss.judge_and_replace('延期天数',1)
ss.select_by_value('延期天数_bool',True)
tt = myModule.format(ss.df,'week_number',['流水号','个性化分析名称','信息负责人','评估反馈时间','预计交付时间','实际交付时间']).df
tt.sort_values(by = '预计交付时间',axis = 0,ascending = False)

,week_number,流水号,个性化分析名称,信息负责人,评估反馈时间,预计交付时间,实际交付时间
105,9月4周,20180917-002,SN170011-04西北农林2个拟南芥样本4个HiC文库测序分析任务单-07,范旭宁,2018-09-17,2018-09-28,2018-10-09 00:00:00


# 7日完成率统计

##  7日完成率趋势图

In [35]:
ss = myModule.format(df4 ,'week_number' ,['流水号','实际周期'])
ss.judge_and_replace('实际周期',7)   # 建议添加小于号
ss.df['流水号'] = list(map(changetype2, ss.df['流水号']))
ss.df['实际周期_bool'] = ss.df['实际周期_bool'].astype(int)      #bool类型装换为数值1/0
pp = myModule.sort_by_index(ss.get_total_ratio([['7日完成率' ,'实际周期_bool' , '流水号','/']]))
myModule.myPlot(pp).plot('line')

/mnt/e/jupyter/bin/supplment_analysis/myModule.py:246: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

## 实际周期>7日项目列表

In [36]:
ss = myModule.format(df4,'week_number',['流水号','个性化分析名称','信息负责人','实际周期'])
ss.select_by_value('week_number',stat_week)
ss.judge_and_replace('实际周期',7)
ss.select_by_value('实际周期_bool',True)
tt = myModule.format(ss.df,'week_number',['流水号','个性化分析名称','信息负责人','实际周期']).df
tt.sort_values(by = '实际周期',axis = 0,ascending = False)   #预计周期从大到小排序

,week_number,流水号,个性化分析名称,信息负责人,实际周期
96,9月4周,20180914-008,PM-NM180047-01内蒙古大学9个单细胞转录组建库测序分析-03,赵倩,10
113,9月4周,20180918-005,YWZJ180310-01浙大昆虫所21个褐飞虱无参普通转录组只分析任务单-01,王霞,8
